In [16]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats

directory = "C:/Users/syversk/Desktop/mss_collocated"
files = os.listdir(directory)
df_list = []
for file in tqdm(files):
    df = pd.read_csv(directory + "/" + file)
    df = remove_extra(df)
    df.to_csv( "C:/Users/syversk/Desktop/mss_collocated" + file ,index=False)
    df_cr1 = reduce_area_of_df(df, "cr1")
    df_cr1.to_csv( "C:/Users/syversk/Desktop/cr1" + file ,index=False)
    df = reduce_area_of_df(df, "cr2")
    df.to_csv( "C:/Users/syversk/Desktop/cr2" + file ,index=False)
    df['mss_anomaly_wind'] = df.apply(lambda row: calculate_mss_anomaly(row.mss, row.wind), axis = 1)
    df['mss_anomaly_delta'] = df.apply(lambda row: calculate_mss_anomaly(row.mss, row.delta), axis = 1)

100%|██████████| 740/740 [12:56<00:00,  1.05s/it]


In [13]:
def reduce_area_of_df(df, cr):
    if cr == "cr1":
        df = df[df.lat <= -10]
        df = df[df.lat >= -25]
        df = df[df.lon >= 105]
        df = df[df.lon <= 120]
    else:
        df = df[df.lat >= 10]
        df = df[df.lat <= 20]
        df = df[df.lon >= 128]
        df = df[df.lon <= 143]
    return df

def remove_extra(df):
    rel_wind_current = np.sqrt((df["era_u10"] - df["oscar_u"])**2 + (df["era_v10"] - df["oscar_v"])**2)
    df.drop(['era_u10', "oscar_u", "era_v10", "oscar_v", "oscar_current"], inplace=True, axis=1)
    df["delta"] = rel_wind_current
    return df

def calculate_mss_anomaly(mss, wind_or_delta):
    if wind_or_delta <= 3.49:
        mss_mod = 0.0035*(1*wind_or_delta + 0.62)
    else:
        mss_mod = 0.0035*(6*np.log(wind_or_delta) + -3.39)
    return (mss-mss_mod)/mss_mod

def grid_data_sub(lat_lon):
    return round(lat_lon*4)/4

def make_data_grided(df):
    df["lon"] = df["lon"].apply(lambda lon: grid_data_sub(lon), axis = 1)
    df["lat"] = df["lat"].apply(lambda lat: grid_data_sub(lat), axis = 1)
    return df

def reduce_based_on_sd():
    z_scores = stats.zscore(df)
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3).all(axis=1)
    return df[filtered_entries]